In [1]:
with open("genes.txt") as io:

    genes = [gene.strip() for gene in io.readlines()[1:]]

In [2]:
import numpy as np

In [3]:
# gene_values = np.full(len(genes), 1 / len(genes))
gene_values = np.random.normal(size=len(genes))

In [4]:
import pandas as pd

In [5]:
gene_x_sample = pd.DataFrame(
    {"Sample 1": gene_values, "Sample 2": gene_values * 10}, index=genes
)

In [6]:
random_gene_set_genes = np.random.choice(genes, size=100, replace=False).tolist()

In [7]:
import kraft

In [8]:
gene_set_genes = {}

for gmt_file_path in [
    "/home/kwat/garden/data/gene_set/msigdb_v6.2/h.all.v6.2.symbols.gmt",
    #     "/home/kwat/garden/data/gene_set/msigdb_v6.2/c1.all.v6.2.symbols.gmt",
    #     "/home/kwat/garden/data/gene_set/msigdb_v6.2/c2.all.v6.2.symbols.gmt",
    #     "/home/kwat/garden/data/gene_set/msigdb_v6.2/c3.all.v6.2.symbols.gmt",
    #     "/home/kwat/garden/data/gene_set/msigdb_v6.2/c5.all.v6.2.symbols.gmt",
    #     "/home/kwat/garden/data/gene_set/msigdb_v6.2/c6.all.v6.2.symbols.gmt",
    #     "/home/kwat/garden/data/gene_set/msigdb_v6.2/c7.all.v6.2.symbols.gmt",
    #     "/home/kwat/garden/data/gene_set/hippo_pathway.gmt",
    #     "/home/kwat/garden/data/gene_set/stem_cell.gmt",
    #     "/home/kwat/garden/data/gene_set/yap_signature.gmt",
]:

    gene_set_genes.update(kraft.read_gmt(gmt_file_path))

In [9]:
from julia.api import Julia

Julia(compiled_modules=False)

from julia import Kraft

In [11]:
Kraft.compute_gene_set_enrichment(
    gene_values,
    genes,
    random_gene_set_genes,
)

(array([-1.72848895e-05, -3.45697791e-05, -5.18546686e-05, ...,
         3.45697792e-05,  1.72848897e-05,  1.44826757e-13]),
 -0.16965289834721012,
 -1147.994405671446)

In [13]:
Kraft.compute_gene_set_enrichment(
    gene_values,
    genes,
    gene_set_genes,
)

RuntimeError: Julia exception: MethodError: no method matching compute_gene_set_enrichment(::Array{Float64,1}, ::Array{String,1}, ::Dict{Any,Any})
Closest candidates are:
  compute_gene_set_enrichment(::Array{Float64,1}, ::AbstractArray{String,1}, !Matched::Array{String,1}; sort_gene_values, gene_index, compute_cumulative_sum) at /home/kwat/.julia/packages/Kraft/DTyoE/src/compute_gene_set_enrichment.jl:31
  compute_gene_set_enrichment(::Array{Float64,1}, ::AbstractArray{String,1}, !Matched::Dict{String,Array{String,1}}; sort_gene_values) at /home/kwat/.julia/packages/Kraft/DTyoE/src/compute_gene_set_enrichment.jl:148
Stacktrace:
 [1] #invokelatest#1(::Base.Iterators.Pairs{Union{},Union{},Tuple{},NamedTuple{(),Tuple{}}}, ::Function, ::Any, ::Any, ::Vararg{Any,N} where N) at ./essentials.jl:697
 [2] invokelatest(::Any, ::Any, ::Vararg{Any,N} where N) at ./essentials.jl:696
 [3] _pyjlwrap_call(::Function, ::Ptr{PyCall.PyObject_struct}, ::Ptr{PyCall.PyObject_struct}) at /home/kwat/.julia/p